In [1]:
import seaborn as sns
sns.set(rc={'axes.facecolor': 'dimgrey', 'grid.color': 'lightgrey'})
import pandas as pd
import torch
import copy
from torch_geometric.loader import NeighborLoader
import pickle

In [2]:
%run utils.ipynb
%run gcn_model.ipynb
%run GAT_model.ipynb

In [3]:
# class 1 is illicit, 2 is licit, unknown is unknown
labels = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
edges = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
nodes = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)

In [4]:
indexes = labels.index[labels["class"] != "unknown"].tolist()
new_labels = labels.replace("unknown", 3)

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
data, graph_info = create_graph(nodes, edges, new_labels, device)
data

Data(x=[203769, 165], edge_index=[2, 234355], y=[203769])

In [7]:
# generate test and train masks
train_p = 0.8
# indices of the illicit and licit transaction nodes
illicit_idx = (data.y == 0).nonzero(as_tuple=True)[0]
# Generate random permutation of indices
illicit_train, illicit_test = torch.utils.data.random_split(illicit_idx, lengths=[train_p, (1-train_p)])

# the same for the licit nodes
licit_idx   = (data.y == 1).nonzero(as_tuple=True)[0]
licit_train, licit_test = torch.utils.data.random_split(licit_idx, lengths=[train_p, (1-train_p)])
# obtain the training and test "masks", i.e. indices for the train and test sets
unlabelled_idx = (data.y == 2).nonzero(as_tuple=True)[0]
unlabelled_train, unlabelled_test = torch.utils.data.random_split(unlabelled_idx, lengths=[train_p, (1-train_p)])


train_mask = torch.cat([torch.tensor(illicit_train), torch.tensor(licit_train)])
test_mask  = torch.cat([torch.tensor(illicit_test), torch.tensor(licit_test)])

train_mask.shape, test_mask.shape

(torch.Size([37253]), torch.Size([9311]))

In [8]:
from torch_geometric.loader import ClusterLoader, RandomNodeLoader, ImbalancedSampler
# from torch_geometric.data import ClusterData
# train_loader = NeighborLoader(
#     data,
#     input_nodes=train_mask,
#     num_neighbors=[-1],
#     batch_size=2048,
# )
#
# test_loader = NeighborLoader(
#     data,
#     input_nodes=test_mask,
#     num_neighbors=[-1],
#     batch_size=2048,
# )
sampler = ImbalancedSampler(data, input_nodes=train_mask)
test_sampler = ImbalancedSampler(data, input_nodes=test_mask)
train_loader = NeighborLoader(data,
                        batch_size=1024, num_neighbors=[20, 1],
                        sampler=sampler,input_nodes=train_mask)

test_loader = NeighborLoader(data,
                              batch_size=1024, num_neighbors=[20, 1],
                              sampler=test_sampler,input_nodes=test_mask)

In [9]:
GCN = GNN_Example(node_dim=data.x.shape[1], edge_dim=1, output_dim=2, hidden_dim=256, n_gnn_layers=3, K=2, dropout_rate=0.1).to(device)

In [ ]:
mega_results_gcn = []
for n in range(10):
    print(n)
    GCN = GNN_Example(node_dim=data.x.shape[1], edge_dim=1, output_dim=2, hidden_dim=256, n_gnn_layers=3, K=2, dropout_rate=0.1).to(device)
    lr = 1e-3
    epochs = 100
    # weights for classes
    weight = torch.tensor([0.5, 0.5]).to(device)
    optimizer = torch.optim.Adam(GCN.parameters(), lr=lr)
    loss_fn = torch.nn.NLLLoss(weight)

    best_model = None
    best_test_loss = 1e10
    best_result = None
    train_losses = []
    test_losses = []

    for epoch in range(1, 1 + epochs):
        # train with random split
        loss = train(GCN, train_loader,train_mask, optimizer, loss_fn, device)
        train_losses.append(loss)
        result, test_loss = test(GCN, test_loader,loss_fn,test_mask, device)
        test_losses.append(test_loss)
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            best_model = copy.deepcopy(GCN)
            best_result = result
        # if epoch % 10 == 0:
        #     print("Epoch: ", epoch)
        #     for item, amount in result.items():
        #             print("{} ({})".format(item, amount))
        #     print("############################")
    state = {"net":best_model.state_dict(), "optimizer":optimizer.state_dict()}
    torch.save(state, "gcn.pt")
    mega_results_gcn.append(best_result)
    print(best_result)

In [12]:
with open("mega_results_gcn", "wb") as fp:   #Pickling
    pickle.dump(mega_results_gcn, fp)

In [13]:
%run GAT_model.ipynb
GAT = GraphAttentionNetwork(node_dim=data.x.shape[1], output_dim=2, hidden_dim=256, n_gnn_layers=3, heads=2, dropout_rate=0.1).to(device)

In [16]:
mega_results_gat = []
for n in range(10):
    print(n)
    GAT = GraphAttentionNetwork(node_dim=data.x.shape[1], output_dim=2, hidden_dim=256, n_gnn_layers=3, heads=2, dropout_rate=0.1).to(device)
    lr = 1e-3
    epochs = 100
    # weights for classes
    weight = torch.tensor([0.5, 0.5]).to(device)
    optimizer = torch.optim.Adam(GAT.parameters(), lr=lr)
    loss_fn = torch.nn.NLLLoss(weight)

    best_model = None
    best_valid_auc = 0
    best_result = None
    losses = []
    best_test_loss = 1e9

    for epoch in range(1, 1 + epochs):
        # train with random split
        train_losses = []
    test_losses = []

    for epoch in range(1, 1 + epochs):
        # train with random split
        loss = train(GAT, train_loader,train_mask, optimizer, loss_fn, device)
        train_losses.append(loss)
        result, test_loss = test(GAT, test_loader,loss_fn,test_mask, device)
        test_losses.append(test_loss)
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            best_model = copy.deepcopy(GAT)
            best_result = result
        # if epoch % 10 == 0:
        #     print("Epoch: ", epoch)
        #     for item, amount in result.items():
        #             print("{} ({})".format(item, amount))
        #     print("############################")
    state = {"net":best_model.state_dict(), "optimizer":optimizer.state_dict()}
    torch.save(state, "gat.pt")
    mega_results_gat.append(best_result)
    print(best_result)


0
{'0': {'precision': 0.7014925373134329, 'recall': 0.9591836734693877, 'f1-score': 0.810344827586207, 'support': 49}, '1': {'precision': 0.979381443298969, 'recall': 0.8260869565217391, 'f1-score': 0.8962264150943396, 'support': 115}, 'accuracy': 0.8658536585365854, 'macro avg': {'precision': 0.840436990306201, 'recall': 0.8926353149955635, 'f1-score': 0.8532856213402733, 'support': 164}, 'weighted avg': {'precision': 0.8963536604130466, 'recall': 0.8658536585365854, 'f1-score': 0.8705666724852025, 'support': 164}}
1
{'0': {'precision': 0.8461538461538461, 'recall': 0.9166666666666666, 'f1-score': 0.8799999999999999, 'support': 60}, '1': {'precision': 0.9541284403669725, 'recall': 0.9122807017543859, 'f1-score': 0.9327354260089685, 'support': 114}, 'accuracy': 0.9137931034482759, 'macro avg': {'precision': 0.9001411432604094, 'recall': 0.9144736842105263, 'f1-score': 0.9063677130044843, 'support': 174}, 'weighted avg': {'precision': 0.916895821672791, 'recall': 0.9137931034482759, 'f1

In [17]:
with open("mega_results_gat", "wb") as fp:   #Pickling
    pickle.dump(mega_results_gat, fp)

In [18]:
%run FeatureAggregationModel.ipynb
SFA = SpatialFeatureAggregator(node_dim=data.x.shape[1], edge_dim=1, output_dim=2, hidden_dim=256, n_gnn_layers=3, K=2, dropout_rate=0.1).to(device)

In [19]:
mega_results_sfa = []
for n in range(10):
    SFA = SpatialFeatureAggregator(node_dim=data.x.shape[1], edge_dim=1, output_dim=2, hidden_dim=256, n_gnn_layers=3, K=2, dropout_rate=0.1).to(device)
    lr = 1e-3
    epochs = 100
    # weights for classes
    weight = torch.tensor([0.5, 0.5]).to(device)
    optimizer = torch.optim.Adam(SFA.parameters(), lr=lr)
    loss_fn = torch.nn.NLLLoss(weight)

    best_model = None
    best_valid_auc = 0
    best_result = None
    losses = []
    best_test_loss = 1e9

    for epoch in range(1, 1 + epochs):
        # train with random split
        train_losses = []
    test_losses = []

    for epoch in range(1, 1 + epochs):
        # train with random split
        loss = train(SFA, train_loader,train_mask, optimizer, loss_fn, device)
        train_losses.append(loss)
        result, test_loss = test(SFA, test_loader,loss_fn,test_mask, device)
        test_losses.append(test_loss)
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            best_model = copy.deepcopy(SFA)
            best_result = result
        if epoch % 10 == 0:
            print("Epoch: ", epoch)
            for item, amount in result.items():
                    print("{} ({})".format(item, amount))
            print("############################")
    state = {"net":best_model.state_dict(), "optimizer":optimizer.state_dict()}
    torch.save(state, "SFA.pt")
    mega_results_sfa.append(best_result)

Epoch:  10
0 ({'precision': 0.803030303030303, 'recall': 0.8688524590163934, 'f1-score': 0.8346456692913385, 'support': 61})
1 ({'precision': 0.9183673469387755, 'recall': 0.8737864077669902, 'f1-score': 0.8955223880597014, 'support': 103})
accuracy (0.8719512195121951)
macro avg ({'precision': 0.8606988249845393, 'recall': 0.8713194333916918, 'f1-score': 0.8650840286755199, 'support': 164})
weighted avg ({'precision': 0.8754675928020875, 'recall': 0.8719512195121951, 'f1-score': 0.8728792182739079, 'support': 164})
############################
Epoch:  20
0 ({'precision': 0.9333333333333333, 'recall': 0.8235294117647058, 'f1-score': 0.8749999999999999, 'support': 51})
1 ({'precision': 0.9181818181818182, 'recall': 0.9711538461538461, 'f1-score': 0.9439252336448598, 'support': 104})
accuracy (0.9225806451612903)
macro avg ({'precision': 0.9257575757575758, 'recall': 0.8973416289592759, 'f1-score': 0.9094626168224298, 'support': 155})
weighted avg ({'precision': 0.92316715542522, 'recall

In [20]:
with open("mega_results_sfa", "wb") as fp:   #Pickling
    pickle.dump(mega_results_sfa, fp)

In [21]:

%run LSTM_GNN.ipynb
TB = TemporalBoy(node_dim=data.x.shape[1], edge_dim=1, output_dim=2, hidden_dim=256, n_gnn_layers=3, K=2, dropout_rate=0.1).to(device)

D:\Python\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [22]:
mega_results_tb =[]
for n in range(10):
    TB = TemporalBoy(node_dim=data.x.shape[1], edge_dim=1, output_dim=2, hidden_dim=256, n_gnn_layers=3, K=2, dropout_rate=0.1).to(device)
    lr = 1e-3
    epochs = 100
    # weights for classes
    weight = torch.tensor([0.5, 0.5]).to(device)
    optimizer = torch.optim.Adam(TB.parameters(), lr=lr)
    loss_fn = torch.nn.NLLLoss(weight)

    best_model = None
    best_valid_auc = 0
    best_result = None
    losses = []
    best_test_loss = 1e9

    for epoch in range(1, 1 + epochs):
        # train with random split
        train_losses = []
    test_losses = []

    for epoch in range(1, 1 + epochs):
        # train with random split
        loss = train(TB, train_loader,train_mask, optimizer, loss_fn, device)
        train_losses.append(loss)
        result, test_loss = test(TB, test_loader,loss_fn,test_mask, device)
        test_losses.append(test_loss)
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            best_model = copy.deepcopy(TB)
            best_result = result
        # if epoch % 10 == 0:
        #     print("Epoch: ", epoch)
        #     for item, amount in result.items():
        #             print("{} ({})".format(item, amount))
        #     print("############################")
    state = {"net":best_model.state_dict(), "optimizer":optimizer.state_dict()}
    torch.save(state, "TB.pt")
    mega_results_tb.append(best_result)

Epoch:  10
0 ({'precision': 0.7959183673469388, 'recall': 0.639344262295082, 'f1-score': 0.7090909090909091, 'support': 61})
1 ({'precision': 0.8405797101449275, 'recall': 0.9206349206349206, 'f1-score': 0.8787878787878787, 'support': 126})
accuracy (0.8288770053475936)
macro avg ({'precision': 0.8182490387459331, 'recall': 0.7799895914650012, 'f1-score': 0.793939393939394, 'support': 187})
weighted avg ({'precision': 0.8260110368257975, 'recall': 0.8288770053475936, 'f1-score': 0.8234321827904715, 'support': 187})
############################
Epoch:  20
0 ({'precision': 0.8309859154929577, 'recall': 0.855072463768116, 'f1-score': 0.8428571428571429, 'support': 69})
1 ({'precision': 0.8958333333333334, 'recall': 0.8775510204081632, 'f1-score': 0.8865979381443299, 'support': 98})
accuracy (0.8682634730538922)
macro avg ({'precision': 0.8634096244131455, 'recall': 0.8663117420881397, 'f1-score': 0.8647275405007364, 'support': 167})
weighted avg ({'precision': 0.8690400888364118, 'recall'

D:\Python\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch:  10
0 ({'precision': 0.8461538461538461, 'recall': 0.6, 'f1-score': 0.7021276595744681, 'support': 55})
1 ({'precision': 0.819672131147541, 'recall': 0.9433962264150944, 'f1-score': 0.8771929824561403, 'support': 106})
accuracy (0.8260869565217391)
macro avg ({'precision': 0.8329129886506936, 'recall': 0.7716981132075471, 'f1-score': 0.7896603210153041, 'support': 161})
weighted avg ({'precision': 0.8287186797521794, 'recall': 0.8260869565217391, 'f1-score': 0.8173880584903517, 'support': 161})
############################
Epoch:  20
0 ({'precision': 0.7901234567901234, 'recall': 0.927536231884058, 'f1-score': 0.8533333333333333, 'support': 69})
1 ({'precision': 0.9494949494949495, 'recall': 0.8468468468468469, 'f1-score': 0.8952380952380952, 'support': 111})
accuracy (0.8777777777777778)
macro avg ({'precision': 0.8698092031425364, 'recall': 0.8871915393654524, 'f1-score': 0.8742857142857142, 'support': 180})
weighted avg ({'precision': 0.8884025439580995, 'recall': 0.877777777

D:\Python\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch:  10
0 ({'precision': 0.7727272727272727, 'recall': 0.5396825396825397, 'f1-score': 0.6355140186915886, 'support': 63})
1 ({'precision': 0.8066666666666666, 'recall': 0.9236641221374046, 'f1-score': 0.8612099644128113, 'support': 131})
accuracy (0.7989690721649485)
macro avg ({'precision': 0.7896969696969697, 'recall': 0.7316733309099721, 'f1-score': 0.7483619915522, 'support': 194})
weighted avg ({'precision': 0.7956451109028428, 'recall': 0.7989690721649485, 'f1-score': 0.7879169511115894, 'support': 194})
############################
Epoch:  20
0 ({'precision': 0.76, 'recall': 0.890625, 'f1-score': 0.8201438848920863, 'support': 64})
1 ({'precision': 0.9263157894736842, 'recall': 0.8301886792452831, 'f1-score': 0.8756218905472637, 'support': 106})
accuracy (0.8529411764705882)
macro avg ({'precision': 0.8431578947368421, 'recall': 0.8604068396226415, 'f1-score': 0.847882887719675, 'support': 170})
weighted avg ({'precision': 0.863702786377709, 'recall': 0.8529411764705882, 'f1

...

Epoch:  10
0 ({'precision': 0.7872340425531915, 'recall': 0.6271186440677966, 'f1-score': 0.6981132075471698, 'support': 59})
1 ({'precision': 0.8135593220338984, 'recall': 0.9056603773584906, 'f1-score': 0.8571428571428572, 'support': 106})
accuracy (0.806060606060606)
macro avg ({'precision': 0.8003966822935449, 'recall': 0.7663895107131435, 'f1-score': 0.7776280323450135, 'support': 165})
weighted avg ({'precision': 0.8041460402801911, 'recall': 0.806060606060606, 'f1-score': 0.800277709711672, 'support': 165})
############################
Epoch:  20
0 ({'precision': 0.7424242424242424, 'recall': 0.8448275862068966, 'f1-score': 0.7903225806451614, 'support': 58})
1 ({'precision': 0.923728813559322, 'recall': 0.8650793650793651, 'f1-score': 0.8934426229508198, 'support': 126})
accuracy (0.8586956521739131)
macro avg ({'precision': 0.8330765279917822, 'recall': 0.8549534756431308, 'f1-score': 0.8418826017979906, 'support': 184})
weighted avg ({'precision': 0.8665784596145687, 'recall'

D:\Python\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch:  10
0 ({'precision': 0.59375, 'recall': 0.3877551020408163, 'f1-score': 0.46913580246913583, 'support': 49})
1 ({'precision': 0.7945205479452054, 'recall': 0.8992248062015504, 'f1-score': 0.8436363636363637, 'support': 129})
accuracy (0.7584269662921348)
macro avg ({'precision': 0.6941352739726028, 'recall': 0.6434899541211834, 'f1-score': 0.6563860830527498, 'support': 178})
weighted avg ({'precision': 0.739252251038941, 'recall': 0.7584269662921348, 'f1-score': 0.7405435125285312, 'support': 178})
############################
Epoch:  20
0 ({'precision': 0.819672131147541, 'recall': 0.78125, 'f1-score': 0.8, 'support': 64})
1 ({'precision': 0.8771929824561403, 'recall': 0.9009009009009009, 'f1-score': 0.8888888888888888, 'support': 111})
accuracy (0.8571428571428571)
macro avg ({'precision': 0.8484325568018407, 'recall': 0.8410754504504505, 'f1-score': 0.8444444444444444, 'support': 175})
weighted avg ({'precision': 0.8561567854061384, 'recall': 0.8571428571428571, 'f1-score': 

D:\Python\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch:  10
0 ({'precision': 0.8125, 'recall': 0.5571428571428572, 'f1-score': 0.6610169491525424, 'support': 70})
1 ({'precision': 0.7832167832167832, 'recall': 0.9256198347107438, 'f1-score': 0.8484848484848485, 'support': 121})
accuracy (0.7905759162303665)
macro avg ({'precision': 0.7978583916083917, 'recall': 0.7413813459268005, 'f1-score': 0.7547508988186955, 'support': 191})
weighted avg ({'precision': 0.7939488521949255, 'recall': 0.7905759162303665, 'f1-score': 0.7797793356405478, 'support': 191})
############################
Epoch:  20
0 ({'precision': 0.75, 'recall': 0.8059701492537313, 'f1-score': 0.776978417266187, 'support': 67})
1 ({'precision': 0.8539325842696629, 'recall': 0.8085106382978723, 'f1-score': 0.8306010928961749, 'support': 94})
accuracy (0.8074534161490683)
macro avg ({'precision': 0.8019662921348314, 'recall': 0.8072403937758018, 'f1-score': 0.803789755081181, 'support': 161})
weighted avg ({'precision': 0.8106811361574429, 'recall': 0.8074534161490683, 'f1

In [23]:
with open("mega_results_tb", "wb") as fp:   #Pickling
    pickle.dump(mega_results_tb, fp)

In [0]:
import numpy as np

def mean_std_results(results):
    f1_0 = []
    f1_1 = []
    f1_avg = []
    for result in results:
        f1_0.append(result["0"]["f1-score"])
        f1_1.append(result["1"]["f1-score"])
        f1_avg.append(result["macro avg"]["f1-score"])
    return (np.mean(f1_0), np.std(f1_0)),(np.mean(f1_1),np.std(f1_1)),(np.mean(f1_avg),np.std(f1_avg)),

In [29]:

mean_std_results(mega_results_gcn)

((0.9388397117002955, 0.017217967851388818),
 (0.9695327511032807, 0.005040981351967361),
 (0.954186231401788, 0.010060960138629123))

In [30]:

mean_std_results(mega_results_gat)

((0.8622748685306652, 0.04235661537995489),
 (0.9081986908288979, 0.03316580341254282),
 (0.8852367796797816, 0.0365638129115355))

In [31]:

mean_std_results(mega_results_sfa)

((0.961785296237881, 0.014817215453102382),
 (0.9801700457635043, 0.0069869538864156845),
 (0.9709776710006928, 0.010681159962797133))

In [32]:

mean_std_results(mega_results_tb)

((0.9492044018231803, 0.013457032458837748),
 (0.9707736878191678, 0.012009375029578807),
 (0.959989044821174, 0.011991497169604065))